In [12]:
import scanpy as sc
import pandas as pd

In [2]:
human_chem = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/extra_human_datasets/chemical_treatment/chemical_treatment_annotated_raw.h5ad")

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
human_chem

AnnData object with n_obs × n_vars = 37257 × 32738
    obs: 'nGene', 'nUMI', 'orig.ident', 'sample', 'percent.mito', 'res.0.5', 'ClusterNames_0.5', 'tSNE_1', 'tSNE_2', 'celltype', 'spikeIn', 'replicate', 'treatment', 'PredictedCell', 'PredictionConf', 'PredictionMax', 'celltype2', 'nGene.peak', 'nGene.doublet', 'INS', 'GCG'

In [5]:
chem_genes = human_chem.var.index.tolist()

In [6]:
chem_genes_just_genes = []
for gene in chem_genes:
    chem_genes_just_genes.append(gene.split("_")[1])

In [8]:
chem_genes = []
for gene in chem_genes_just_genes:
    chem_genes.append(gene.split(".")[0])

In [10]:
# Save to a .txt file
with open("genes_in_the_dataset_actual_human_chem.txt", "w") as file:
    for item in chem_genes:
        file.write("%s , " % item)

In [26]:
genes = pd.read_table("make_chem_data/genes_in_the_dataset_actual_human_chem.txt")

In [27]:
genes = genes[genes["Mouse homology type"] == "ortholog_one2one"]

In [30]:
human_chem.var["name"] = chem_genes

In [35]:
human_chem = human_chem[:,~human_chem.var.name.duplicated()]

In [37]:
genes = genes[~genes["Gene name"].duplicated()]

In [42]:
human_chem.var = human_chem.var.merge(genes, left_on="name", right_on="Gene name", how = "left")

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:798: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [48]:
human_chem.var.index = human_chem.var["name"]

In [51]:
human_chem = human_chem[:,~human_chem.var["Mouse gene stable ID"].isna()]

In [56]:
var = pd.DataFrame(index = human_chem.var["Mouse gene stable ID"])

In [57]:
var.index = var.index.rename("EID")

In [59]:
human_chem.var = var

In [61]:
human_chem.obs["disease"] = "healthy"
human_chem.obs["organism"] = "Homo sapiens"
human_chem.obs["dataset"] = "chem"

In [66]:
human_chem.obs.rename({"celltype2": "cell_type"}, axis = 1, inplace=True)

In [67]:
human_chem.obs = human_chem.obs[['disease', 'dataset', 'organism', 'cell_type']]

In [72]:
human_chem.write_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/extra_human_datasets/chemical_treatment/chemical_treatment_annotated_raw_orth.h5ad")

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c
/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c
/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [75]:
sc.pp.normalize_per_cell(human_chem)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[key_n_counts] = counts_per_cell


In [76]:
sc.pp.log1p(human_chem)

In [77]:
human_chem.write_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/extra_human_datasets/chemical_treatment/chemical_treatment_annotated_norm_log1p_orth.h5ad")